In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')

Target or dependant variable is imbalanced.

In [ ]:
# Compute the length of each essay and add it as a new feature
train['full_text'] = train['full_text'].str.lower()
train['full_text'] = train['full_text'].str.replace(r'[^\w\s]', '')
train.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,4),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True)
vectorizer= vec.fit(train['full_text'])
X = vectorizer.transform(train['full_text'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, train['score'], test_size=0.2, random_state=42)

In [ ]:
# Define LightGBM parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 150,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
import lightgbm as lgb
# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3),
    ])

In [ ]:
from sklearn.metrics import mean_squared_error
# Evaluate the model
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
print(f"Root Mean Squared Error (RMSE) on test set: {rmse}")

In [ ]:
test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

In [ ]:
test_ip = vectorizer.transform(test['full_text'])

In [ ]:
preds = model.predict(test_ip)

In [ ]:
preds = np.round(preds)

In [ ]:
out = pd.DataFrame()
out['essay_id'] = test['essay_id']
out['score'] = preds

In [ ]:
out.to_csv('submission.csv', index = False)